We are going to edit the Maze example to work on the menu search scenario. Here we have three classes:
1. Menu class, which implements the Environment class in PyBrain
2. MDPMenuTask class, which implements the Task class in PyBrain

First import the libraries you need for the Menu environment definition. Note that we import the class Environment from pybrain.rl.environments.environment. The Menu class inherits from Environment and implements the abstract initialization method, reset(), performAction() and getSensors(). It is a good idea to take a closer look at these methods, because they define how our environment behaves.

In [ ]:
from random import random, choice
from scipy import zeros
import numpy as np

from pybrain.utilities import Named
from pybrain.rl.environments.environment import Environment

In [ ]:
class Menu(Environment, Named):
    """ 2D mazes, with actions being the direction of movement (N,E,S,W)
    Note that in this example it is a menu represeted as a one dimesional array.
    Hence, the user can traverse only in left (west) or right (east) directions
    The observations can be noisy.
    """

    # table of booleans
    mazeTable = None

    # single goal
    goal = None

    # current state
    perseus = None

    # list of possible initial states
    initPos = None

    # directions the agent can move. 
    N = (1, 0)
    S = (-1, 0)
    E = (0, 1)
    W = (0, -1)

    #note that the action space contains only east or west movements
    

The code below (allActions = [E,W]) is one of the first codes we need to change from the maze example. In a Maze you can move in all 4 directions (N,S,E,W). However, in the menu your movement is restricted to two directions (up and down/ left and right). If we define the menu as a 1-dimensional array then the possible actions as E (moving to right) and W (moving to left). 

In [ ]:
    allActions = [E,W]

We set stochasticity to 0

In [ ]:
    # stochasticity
    stochAction = 0.
    stochObs = 0.
    
    

The __init__ method initiales parameters of the environemnt before training. Topology is the structure of the menu, and the goal defines the location of the target item. We assume that all the items in this menu are irrelevant (relevance = 0) except for the target item, where relevance = 1. This is the simplest menu we could think of, a more complex menu would be a semantic menu, where there are items partially relevant to the target. 

In [ ]:
    def __init__(self, topology, goal, **args):
        allActions = range(0,np.size(topology),1)
        print allActions

        self.setArgs(**args)
        self.mazeTable = topology
        self.goal = goal
        if self.initPos == None:
            self.initPos = self._freePos()
            self.initPos.remove(self.goal)
        self.reset()

        

The reset function is called whenever a new training trial is started. 

In [ ]:
    def reset(self):
        """ return to initial position (stochastically): """
        self.bang = False
        self.perseus = choice(self.initPos)

_freePos and _moveInDir functions are specific to this example. _moveInDir finds the gaze location when the action coordinates --dir-- and  the current position coordinates --pos -- are given. 

In [ ]:
    def _freePos(self):
        """ produce a list of the free positions. """
        res = []
        for i, row in enumerate(self.mazeTable):
            for j, p in enumerate(row):
                if p == False:
                    res.append((i, j))
        return res

    def _moveInDir(self, pos, dir):
        """ the new state after the movement in one direction. """
        return (pos[0] + dir[0], pos[1] + dir[1])
    

performAction is an important function. This is called whenever the agent choses an action. First we compute the current location --tmp-- by using _moveInDir function. Then we check whether the new position is a wall or not. If it is a wall, the agent bangs on the wall (bang = True). Otherwise we set the coordinates of the new position (perseus) to the new location and set bang to False. 

In [ ]:
    def performAction(self, action):

        if self.stochAction > 0:
            if random() < self.stochAction:
                action = choice(list(range(len(self.allActions))))

        tmp = self._moveInDir(self.perseus, self.allActions[action])
        
        if self.mazeTable[tmp] == False:
            self.perseus = tmp
            self.bang = False
        else:
            self.bang = True
            

the getSensort function returns the state_id -- which is a unique id for each specific state the agent can be in. 

In [ ]:
    def getSensors(self):
        obs = zeros(2)
        for i, a in enumerate(Maze.allActions):
            obs[i] = self.mazeTable[self._moveInDir(self.perseus, a)]
        if self.stochObs > 0:
            for i in range(len(obs)):
                if random() < self.stochObs:
                    obs[i] = not obs[i]
        return obs

    def __str__(self):
        """ Ascii representation of the maze, with the current state """
        s = ''
        for r, row in reversed(list(enumerate(self.mazeTable))):
            for c, p in enumerate(row):
                if (r, c) == self.goal:
                    s += '*'
                elif (r, c) == self.perseus:
                    s += '@'
                elif p == True:
                    s += '#'
                else:
                    s += ' '
            s += '\n'
        return s

In [ ]:
from pybrain.rl.environments import Task
from scipy import array

In [ ]:
class MDPMenuTask(Task):
    """ This is a MDP task for the MenuEnvironment. The state is fully observable,
        giving the agent the current position of perseus. Reward is given on reaching
        the goal, otherwise no reward. """

    def getReward(self):
        """ compute and return the current reward (i.e. corresponding to the last action performed)
        Note that now we give a negative reward for visiting irrelevant menu items."""
        if self.env.goal == self.env.perseus:
            self.env.reset()
            reward = 1.
        else:
            reward = -1.
        return reward

    def performAction(self, action):
        """ The action vector is stripped and the only element is cast to integer and given
            to the super class.
        """
        Task.performAction(self, int(action[0]))


    def getObservation(self):
        """ The agent receives its position in the menu, to make this a fully observable
            MDP problem.
        """
        obs = array([self.env.perseus[0] * self.env.mazeTable.shape[0] + self.env.perseus[1]])
        return obs



In [ ]:
from scipy import *

In [ ]:

from pybrain.rl.learners.valuebased import ActionValueTable
from pybrain.rl.agents import LearningAgent
from pybrain.rl.learners import Q
from pybrain.rl.experiments import Experiment


In [ ]:
import pylab
pylab.gray()
pylab.ion()

In [ ]:
# create the menu with 7 items
matrix_size = 7
envmatrix = array([[0,0,0,0,0,0,1]])

env = Menu(envmatrix,(0,2))

# create task
task = MDPMenuTask(env)



# create value table and initialize with ones
table = ActionValueTable(matrix_size, 2)
table.initialize(1.)


# create agent with controller and learner - use SARSA(), Q() or QLambda() here
learner = Q()

# standard exploration is e-greedy, but a different type can be chosen as well, such as:
# learner.explorer = BoltzmannExplorer()

# create agent
agent = LearningAgent(table, learner)

# create experiment
experiment = Experiment(task, agent)

# prepare plotting
pylab.gray()
pylab.ion()

for i in range(100):
    # interact with the environment (here in batch mode)
    experiment.doInteractions(matrix_size)
    agent.learn()
    agent.reset()
    # and draw the table
    pylab.pcolor(table.params.reshape(matrix_size,2).max(1).reshape(matrix_size,1))
    pylab.draw()
    pylab.ion()
    pylab.show()
print "training complete"
